### a) Leitura do arquivo de entrada em estruturas de dados apropriadas

In [11]:
from math import inf

from contratos import Contratos


def ler_contratos_de_arquivo(caminho: str) -> Contratos:
  with open('entrada.txt', 'r') as arquivo:
    # Lendo a primeira linha do arquivo texto
    n, m, t = arquivo.readline().split()

    t_taxa = float(t) # Valor da taxa de mudança de fornecedor
    n_meses = int(n) # Quantidade de meses
    m_fornecedores = int(m) # Quantidade de fornecedores

    print(f'(n, m, t) = ({n_meses}, {m_fornecedores}, {t_taxa:.2f})')
    print()

    # Lendo o restante das linhas do arquivo e gerando a matriz base
    # m * n * n
    A = [[[inf for k in range(n_meses)] for j in range(n_meses)] for i in range(m_fornecedores)]

    print(f'Contratos armazenados em matriz: {len(A)}x{len(A[0])}x{len(A[0][0])}')
    print(f'Tamanho total da matriz: {len(A) * len(A[0]) * len(A[0][0])}')

    for linha in arquivo:
        dados = linha.split(' ');
        i_fornecedor, j_inicio, k_fim = map(lambda i: int(i), dados[:-1])
        A[i_fornecedor - 1][j_inicio - 1][k_fim - 1] = float(dados[-1])
    
    return Contratos(A, t_taxa)

c = ler_contratos_de_arquivo('entrada.txt')

(n, m, t) = (120, 100, 100.00)

Contratos armazenados em matriz: 100x120x120
Tamanho total da matriz: 1440000


**b)** Complexidade da estrutura de dados = $Θ(m * n^2)$

### c) Criar uma função eficiente que retorna o contrato individual, referente ao período completo de n meses, que possui o menor valor

In [12]:
from contratos import Contrato


def get_menor_contrato_periodo(c: Contratos, mes_inicial: int, mes_final: int) -> Contrato:
    menor_contrato = Contrato(c.num_fornecedores, c.num_meses, c.num_meses, inf)
    for forn in range(c.num_fornecedores):
        contrato = c.get_contrato(forn, mes_inicial, mes_final)
        if contrato.valor < menor_contrato.valor:
            menor_contrato = contrato
    return menor_contrato

menor_contrato_periodo_completo = get_menor_contrato_periodo(c, 0, c.num_meses - 1)
print(f'Menor contrato de período completo: {menor_contrato_periodo_completo}')


Menor contrato de período completo: Contrato(fornecedor=61, mês inicial=1, mês final=120, valor=687.10)


**d)** Complexidade de (c): $Θ(m)$

### e) Criar uma função eficiente que retorna o contrato individual de menor valor do mercado, independente do período a que se refere

=resolvido como instância de pior caso do item g

**f)** =complexidade de pior caso do item h

### g) Criar uma função eficiente que retorna o contrato individual, referente ao período completo de x meses (passado como parâmetro, x ≤ n), que possui o menor valor

In [13]:
def menor_contrato_periodo_completo(c: Contratos, x_num_meses: int) -> Contrato:
    menor_contrato = Contrato(c.num_fornecedores, c.num_meses, c.num_meses, inf)

    offset_periodo = x_num_meses - 1
    for inicio in range(c.num_meses - offset_periodo):
        fim = inicio + offset_periodo
        contrato = get_menor_contrato_periodo(c, inicio, fim)
        if contrato.valor < menor_contrato.valor:
            menor_contrato = contrato
    return menor_contrato

x_periodo = c.num_meses
menor_contrato = menor_contrato_periodo_completo(c, x_periodo)
print(f'O menor contrato de periodo completo de {x_periodo} meses é: {menor_contrato}')

O menor contrato de periodo completo de 120 meses é: Contrato(fornecedor=61, mês inicial=1, mês final=120, valor=687.10)


Instância que responde ao item e:

In [14]:
x_periodo = 1
menor_contrato = menor_contrato_periodo_completo(c, x_periodo)
print(f'O menor contrato de periodo completo de {x_periodo} meses é: {menor_contrato}')

O menor contrato de periodo completo de 1 meses é: Contrato(fornecedor=96, mês inicial=4, mês final=4, valor=10.00)


**h)**

Melhor caso (todo o período): $Θ(m)$

Pior caso (período de 1 mês): $Θ(m * n)$

Caso médio: $O(m * n)$

### i) Criar um método que sugere quais contratos de energia devem ser contratados para os próximos n meses

In [15]:
def menores_contratos_periodo_completo(c: Contratos) -> list:
    menor_contrato = menor_contrato_periodo_completo(c, 1)                      # Θ(m * n)
    menor_contrato_completo = menor_contrato_periodo_completo(c, c.num_meses)   # Θ(m)
    if menor_contrato.valor * c.num_meses > menor_contrato_completo.valor:
        return [menor_contrato_completo]                                        # Melhor caso: Θ(m * n)

    inicio = 0
    taxa_mud = c.taxa_mudanca
    intervalo = 1
    menor_total = menor_contrato.valor
    menores_contratos = [menor_contrato_completo]
    for inicio in range(c.num_meses, intervalo):                                # f_1 <- Θ(n / i)
        fornecedor = -1
        valor_total = 0.0
        solucao_atual = []
        print(f'Início {inicio}:')                                              # Sn = n * (a1 + an) / 2
        for i in range(inicio, c.num_meses, intervalo):                         # f_2 <- O(f_1 * (Sum(i, ..., n - 2i, n - i, n))
            pfim = i + intervalo - 1                                            # Sn <- ((n - i)*(n + i)) / 2 -- Θ((n^2 + i^2) / 2)
            if pfim >= c.num_meses:
                pfim = c.num_meses - 1
            contrato = get_menor_contrato_periodo(c,                            # Θ(f_2 * m)
                0 if valor_total <= 0 else i,
                pfim
            )
            valor_total += contrato.valor
            if fornecedor > -1 and fornecedor != contrato.fornecedor:
                valor_total += taxa_mud
            print(f'{contrato} - Valor Total: {valor_total:.2f}'.format(contrato, valor_total))
            if valor_total > menor_total:
                print(f'O valor da lista de contratos atual {valor_total:.2f} ultrapassou o limite {menor_total:.2f}.')
                break
            solucao_atual.append(contrato)
            fornecedor = contrato.fornecedor
        if valor_total < menor_total:
            print(f'Menor lista de contratos encontrada com valor total de {valor_total:.2f}')
            menor_total = valor_total
            menores_contratos = solucao_atual
    return menores_contratos

menores_contratos = menores_contratos_periodo_completo(c)
print(f'Selecione os seguintes contratos para este período de {c.num_meses} meses: {menores_contratos}')

Selecione os seguintes contratos para este período de 120 meses: [Contrato(fornecedor=61, mês inicial=1, mês final=120, valor=687.10)]


j) Apresentar a complexidade da função descrita no item anterior, fazendo uso de notação assintótica e tendo como parâmetros somente a quantidade n de meses e a quantidade m de fornecedores.

| Melhor caso  |  Pior caso   | Caso médio   |
|:------------:|:------------:|:------------:|
|  $Θ(m * n)$  | $Θ(m * n^3)$ | $Θ(m * n^3)$ |